# Inheritance

## Class Attributes

In the work covered so far, we have used **instance-level data**, eg the data is passed
to the class on instantiation. Eg -   
`foo = MyClass("Some data")`

If you would like to ensure some values are passed to all instances of the class, then
you can use a **Class Attribute** to achieve this. Eg -   
```
def MyClass:
    CONSTANT_VAR = 1
    ...

```

To then use this class attribute in `__init__()` or a method, you need to reference it
with the class name like:

`MyClass.CONSTANT_VAR`.


## Class Methods

You can also define 'global' methods, methods that are common to every instance of that
class. There are some pros and cons discussed. The main con is that class methods cannot
access instance level data. 

To initiate a class method, use a decorator:

```
def MyClass:

    @classmethod
    def some_method(cls, arg):
        # Do something
        return cls(some_returnable_object)

```

Note instead of `self` we now refer to `cls`. This is because `self` refers to the
instance, not the class. 

To use the class method we need to:

`MyClass.some_method(arg)`
